In [1]:
import os

import numpy as np
import matplotlib.pyplot as plt
import open3d as o3d
import cv2

import json

current_directory = os.getcwd()

## Load ScanNet scene

In [2]:
scan = "scene0000_00"

scan_path = os.path.join(current_directory, "data/scans", scan)
scene_mesh_path = os.path.join(scan_path, scan + "_vh_clean_2.ply")
scene_mesh = o3d.io.read_triangle_mesh(scene_mesh_path)
segmentation_mesh_path = os.path.join(scan_path, scan + "_vh_clean_2.labels.ply")
segmentation_mesh = o3d.io.read_triangle_mesh(segmentation_mesh_path)

# o3d.visualization.draw_geometries([scene_mesh])

## Select a class

The class must be chosen among one of the raw_category in the [scannetv2-labels.combinesd.tsv](data/scannetv2-labels.combined.tsv)

In [3]:
with open(os.path.join(scan_path, scan + '_vh_clean.aggregation.json')) as f:
    aggregation_data = json.load(f)

all_classes = set()
for seg_group in aggregation_data['segGroups']:
    all_classes.add(seg_group['label'])

print("All labels in the aggregation data: ", all_classes)

All labels in the aggregation data:  {'toaster oven', 'dish rack', 'couch', 'wall', 'table', 'shower', 'tissue box', 'doorframe', 'refrigerator', 'kitchen cabinets', 'floor', 'tv', 'bed', 'clock', 'backpack', 'mirror', 'sink', 'stool', 'bicycle', 'desk', 'door', 'toaster', 'shoes', 'curtain', 'guitar', 'cabinet', 'nightstand', 'pillow', 'ceiling', 'laundry basket', 'trash can', 'coffee table', 'microwave', 'shelf', 'toilet', 'object', 'kitchen counter', 'guitar case', 'scale', 'window'}


In [4]:
class_name = 'bed'

with open(os.path.join(scan_path, scan + '_vh_clean.aggregation.json')) as f:
    aggregation_data = json.load(f)

object_ids = []
for seg_group in aggregation_data['segGroups']:
    if seg_group['label'] == class_name:
        object_ids.append(seg_group['objectId'])

if not object_ids:
    print("No objects found for class:", class_name)
else:
    print(f"Found {len(object_ids)} objects for class {class_name}: {object_ids}")


Found 1 objects for class bed: [37]


## Select an object

In [5]:
object_id = 37
for seg_group in aggregation_data['segGroups']:
    if seg_group['objectId'] == object_id:
        selected_instance_segments = seg_group['segments']

# Load all vertices
with open(os.path.join(scan_path, scan + '_vh_clean_2.0.010000.segs.json')) as f:
    segmentation_data = json.load(f)

# Load instance vertices
all_vertices = np.array(segmentation_data['segIndices'])

# Get indices of instance vertices
instance_vertices_mask = np.isin(all_vertices, selected_instance_segments)
instance_vertex_indices = np.where(instance_vertices_mask)[0]

# Filter faces of the mesh: included if all its vertices are part of the instance
faces = np.asarray(scene_mesh.triangles)
face_mask = np.all(np.isin(faces, instance_vertex_indices), axis=1)
instance_faces = faces[face_mask] # Triplets of vertex indices forming each triangle (the indices refer to scene vertices - all of them)
instance_vertices = np.asarray(scene_mesh.vertices)[instance_vertex_indices] # Coordinates of each vertex
vertex_remap = {scene_idx: instance_idx for instance_idx, scene_idx in enumerate(instance_vertex_indices)}
instance_faces = np.vectorize(vertex_remap.get)(instance_faces) # Triplets of vertex indices forming each triangle (the indices refer to instance vertices - masked)

# Create the mesh for the selected instance
instance_mesh = o3d.geometry.TriangleMesh()
instance_mesh.vertices = o3d.utility.Vector3dVector(instance_vertices)
instance_mesh.triangles = o3d.utility.Vector3iVector(instance_faces)

# o3d.visualization.draw_geometries([instance_mesh])

## Get camera parameters

### Export camera parameters
**Note:** This step should be performed only at the beginning of the process.

In [12]:
reader_directory = os.path.join(current_directory, "scripts/SensReader")
sens_file = os.path.join(scan_path, scan + ".sens")
output_directory = os.path.join(current_directory, "outputs/reader/"+scan)

os.system(f"python {os.path.join(reader_directory, 'reader.py')} --filename {sens_file} --output_path {output_directory} --export_depth_images --export_color_images --export_poses --export_intrinsics --export_depth_images")

Namespace(filename='/Users/lara/Desktop/Making-CLIP-features-multiview-consistent/data/scans/scene0000_00/scene0000_00.sens', output_path='/Users/lara/Desktop/Making-CLIP-features-multiview-consistent/outputs/reader/scene0000_00', export_depth_images=True, export_color_images=True, export_poses=True, export_intrinsics=True)
loading /Users/lara/Desktop/Making-CLIP-features-multiview-consistent/data/scans/scene0000_00/scene0000_00.sens...loaded!
exporting 5578  depth frames to /Users/lara/Desktop/Making-CLIP-features-multiview-consistent/outputs/reader/scene0000_00/depth
exporting 5578 color frames to /Users/lara/Desktop/Making-CLIP-features-multiview-consistent/outputs/reader/scene0000_00/color
exporting 5578 camera poses to /Users/lara/Desktop/Making-CLIP-features-multiview-consistent/outputs/reader/scene0000_00/pose
exporting camera intrinsics to /Users/lara/Desktop/Making-CLIP-features-multiview-consistent/outputs/reader/scene0000_00/intrinsic


0

### Load camera parameters

In [6]:
output_directory = os.path.join(current_directory, "outputs/reader/"+scan)
pose_directory = os.path.join(output_directory, "pose")
originals_directory = os.path.join(output_directory, "color")
depth_directory = os.path.join(output_directory, "depth")
pose_files = [f for f in os.listdir(pose_directory) if f.endswith('.txt')]
frame_indices = [int(f.split('.')[0]) for f in pose_files]
intrinsics = np.loadtxt(os.path.join(output_directory, "intrinsic", "intrinsic_color.txt"))  # Camera intrinsics
# extrinsics = np.loadtxt(os.path.join(output_directory, "intrinsic", "extrinsic_color.txt"))  # Camera estrinsic
camera_intrinsics = intrinsics[:3, :3]

## Select frames that contain the specific object

### Select minimum_vertices in an image
Select the percentage of the object you want in the frame

In [7]:
percentage = 60

total_vertices = instance_vertices.shape[0]
minimum_vertices = int(percentage / 100 * total_vertices)

total_faces = instance_faces.shape[0]
minimum_faces = int(percentage / 100 * total_vertices)

print(minimum_vertices)

1772


## Get masked images

### Select margin of error for depth

In [110]:
# Tolerance on occlusions in meters
error = 1

### Get masked images

In [113]:
def get_2d_homogeneous(points_3d, intrinsic, extrinsic):
    inv_extrinsic = np.linalg.inv(extrinsic)
    points_3d_homogeneous = np.hstack([points_3d, np.ones((points_3d.shape[0], 1))])
    points_camera = np.dot(inv_extrinsic, points_3d_homogeneous.T).T
    points_camera_xyz = points_camera[:, :3] # Extract only the x, y, z components because the intrinsic matrix is 3x3
    points_2d_homogeneous = np.dot(intrinsic, points_camera_xyz.T).T # the third value is the depth
    return points_2d_homogeneous

def draw_triangle(img, vertices, color):
    cv2.fillConvexPoly(img, vertices, color)

def get_depth_map(frame_index, h, w):
    depth_path = os.path.join(depth_directory, f"{frame_index}.png")
    depth_original = cv2.imread(depth_path, cv2.IMREAD_ANYDEPTH) / 1000
    # Depth frames are captured at a resolution of 640 × 480 and color at 1296 × 968 pixels
    depth = cv2.resize(depth_original, (h, w))
    return depth

def get_original_frame(frame_index):
    original_path = os.path.join(originals_directory, f"{frame_index}.jpg")
    return cv2.cvtColor(cv2.imread(original_path), cv2.COLOR_BGR2RGB)

def get_camera_extrinsics(frame_index):
    extrinsics_path = os.path.join(pose_directory, f"{frame_index}.txt")
    return  np.loadtxt(extrinsics_path)

# Get projected points and z coordinate
def get_xyz(vertices, intrinsics, extrinsics):
    points_2d = get_2d_homogeneous(vertices, intrinsics, extrinsics)
    points_2d_xy = (points_2d[:, :2] / points_2d[:, 2].reshape(-1, 1)).astype(int)
    points_2d[:, :2] = points_2d_xy
    return points_2d

def get_occluded_vertices_idx(depth, points):
    total_vertices = points.shape[0]
    image_width, image_height = depth.shape
    occluded_idx = []
    valid_vertices = 0
    for vertex in range(total_vertices):
        point = points[vertex]
        # Check if the vertex falls into the image
        if (point[0] >= 0)& (point[0] < image_width) & (point[1] >= 0) & (point[1] < image_height):
            # Check if the vertex is in front of the camera
            if point[2] > 0:
                valid_vertices += 1
                # Check if the vertex is occluded
                if depth[int(point[0]), int(point[1])] + error < point[2]:
                    occluded_idx.append(vertex)
    return valid_vertices, occluded_idx

def get_camera_for_rendering(camera_extrinsics, camera_intrinsics, h, w):
    camera = o3d.camera.PinholeCameraParameters()
    intrinsic = o3d.camera.PinholeCameraIntrinsic()
    intrinsic.set_intrinsics(width=w, height=h, fx=camera_intrinsics[0][0], fy=camera_intrinsics[1][1], cx=camera_intrinsics[0][2], cy=camera_intrinsics[1][2])
    camera.intrinsic = intrinsic
    camera.extrinsic = np.array(np.linalg.inv(camera_extrinsics))
    return camera

def render_image(mesh, camera, w, h):
    vis = o3d.visualization.Visualizer()
    vis.create_window(visible=False, width = w, height = h)
    mesh.paint_uniform_color([0, 0, 0])  # Black mesh
    vis.add_geometry(mesh)
    vis.get_view_control().convert_from_pinhole_camera_parameters(camera, True)
    vis.get_render_option().background_color = np.array([1, 1, 1])  # White background
    vis.get_render_option().light_on = False
    vis.poll_events()
    vis.update_renderer()
    image = vis.capture_screen_float_buffer(do_render=False)
    vis.destroy_window()
    return image

def process_frame(frame_index):
    original_frame = get_original_frame(frame_index)
    image_height, image_width = original_frame.shape[:2]
    camera_extrinsics = get_camera_extrinsics(frame_index)
    depth = get_depth_map(frame_index, image_height, image_width)

    points_2d = get_xyz(np.asarray(instance_mesh.vertices), camera_intrinsics, camera_extrinsics)

    valid_vertices, vertices_to_remove = get_occluded_vertices_idx(depth, points_2d)
    
    if valid_vertices - len(vertices_to_remove) < minimum_vertices:
        return
    
    vertices_to_remove = set(vertices_to_remove)
    all_faces = np.asarray(instance_mesh.triangles)
    faces_to_keep = [face for face in all_faces if not any(vertex in vertices_to_remove for vertex in face)]

    # Re-index faces: create a mapping from old to new vertex indices
    remaining_vertices_indices = list(set(range(len(instance_mesh.vertices))) - vertices_to_remove)
    new_index_map = {old_idx: new_idx for new_idx, old_idx in enumerate(remaining_vertices_indices)}
    # Update face indices based on new vertex indices
    new_faces = [[new_index_map[vertex] for vertex in face] if all(vertex in new_index_map for vertex in face) else None for face in faces_to_keep]
    new_faces = [face for face in new_faces if face is not None]
    # Get new mesh
    new_vertices = np.asarray(instance_mesh.vertices)[remaining_vertices_indices]
    new_mesh = o3d.geometry.TriangleMesh()
    new_mesh.vertices = o3d.utility.Vector3dVector(new_vertices)
    new_mesh.triangles = o3d.utility.Vector3iVector(new_faces)

    # Rendering
    camera = get_camera_for_rendering(camera_extrinsics, camera_intrinsics, image_height, image_width)
    image = render_image(new_mesh, camera, image_width, image_height)
    image = np.asarray(image)

    # Masking
    mask = (image[:, :, 0] == 0)
    masked_frame_wrong_colors = np.zeros_like(original_frame)
    masked_frame_wrong_colors[mask] = original_frame[mask]
    masked_frame = cv2.cvtColor(masked_frame_wrong_colors, cv2.COLOR_RGB2BGR)

    return masked_frame


selected_frames = []
masked_images_directory = os.path.join(current_directory, "outputs/masked_images/project_without_occlusion/"+scan, f"{object_id}")
if not os.path.exists(masked_images_directory):
    os.makedirs(masked_images_directory)

for frame_index in frame_indices:
    masked_frame = process_frame(frame_index)
    if masked_frame is not None:
        output_path = os.path.join(masked_images_directory, f"{frame_index:05d}.jpg")
        cv2.imwrite(output_path, masked_frame)